## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-29-08-44-18 +0000,nyt,"A Hong Kong Fire Survivor’s Escape, in His Own...",https://www.nytimes.com/2025/11/28/world/asia/...
1,2025-11-29-08-39-13 +0000,nyt,National Guard Soldier Dies After Being Shot i...,https://www.nytimes.com/2025/11/28/us/national...
2,2025-11-29-08-37-57 +0000,nyt,Deadly Hong Kong Fire Is a Test of Beijing’s R...,https://www.nytimes.com/2025/11/29/world/asia/...
3,2025-11-29-08-09-13 +0000,nypost,"Trump, Venezuelan President Maduro spoke last ...",https://nypost.com/2025/11/29/us-news/trump-ve...
4,2025-11-29-08-00-17 +0000,nyt,N.Y. Law Could Set Stage for A.I. Regulation’s...,https://www.nytimes.com/2025/11/29/nyregion/pe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,36
62,shooting,22
7,national,15
8,guard,15
1,kong,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
128,2025-11-28-18-27-18 +0000,startribune,"In social media posts, Trump targets Somali co...",https://www.startribune.com/in-social-media-po...,105
92,2025-11-28-21-31-00 +0000,wsj,President Trump said he was revoking all execu...,https://www.wsj.com/politics/trump-says-he-is-...,102
170,2025-11-28-15-48-03 +0000,nyt,Trump Uses National Guard Shooting to Cast Sus...,https://www.nytimes.com/2025/11/27/us/politics...,92
12,2025-11-29-06-53-08 +0000,nypost,Trump says he plans to pardon former Honduran ...,https://nypost.com/2025/11/29/us-news/trump-pl...,86
56,2025-11-29-00-44-32 +0000,nypost,Trump halts asylum cases following DC shooting...,https://nypost.com/2025/11/28/us-news/trump-ha...,85


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
128,105,2025-11-28-18-27-18 +0000,startribune,"In social media posts, Trump targets Somali co...",https://www.startribune.com/in-social-media-po...
92,66,2025-11-28-21-31-00 +0000,wsj,President Trump said he was revoking all execu...,https://www.wsj.com/politics/trump-says-he-is-...
125,61,2025-11-28-18-45-45 +0000,nypost,Dog saves retiree from narrowly being killed i...,https://nypost.com/2025/11/28/world-news/elder...
164,58,2025-11-28-16-08-10 +0000,nypost,Zelensky’s top aide Andriy Yermak resigns afte...,https://nypost.com/2025/11/28/world-news/zelen...
100,51,2025-11-28-20-44-05 +0000,nypost,Brother of alleged DC gunman Rahmanullah Lakan...,https://nypost.com/2025/11/28/us-news/brother-...
65,51,2025-11-28-23-50-12 +0000,nypost,Treasury Secretary Scott Bessent moves to ‘cut...,https://nypost.com/2025/11/28/us-news/treasury...
41,48,2025-11-29-02-22-56 +0000,nypost,4 arrested as rowdy anti-Israel protesters dis...,https://nypost.com/2025/11/28/us-news/anti-isr...
132,38,2025-11-28-18-15-31 +0000,nyt,Israeli Forces Kill Two Palestinians in West B...,https://www.nytimes.com/2025/11/28/world/middl...
99,34,2025-11-28-20-46-46 +0000,latimes,Will California's new K-12 antisemitism law ma...,https://www.latimes.com/california/story/2025-...
12,28,2025-11-29-06-53-08 +0000,nypost,Trump says he plans to pardon former Honduran ...,https://nypost.com/2025/11/29/us-news/trump-pl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
